In [2]:
import pickle
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.init as init

#opening the batches.meta file for info on label names
with open('batches.meta', 'rb') as f:
    meta = pickle.load(f)

# Combining all the data into one dataset
def load_cifar_batches(batch_folder, num_batches=5):

    images = []
    labels = []

    #looping through the folder and unpickling the datasets
    for i in range(1, num_batches + 1):
        batch_file = os.path.join(batch_folder, f"data_batch_{i}")
        with open(batch_file, 'rb') as f:
            batch = pickle.load(f, encoding = 'bytes')

        #extracting and processing data
        batch_images = batch[b'data'].reshape(10000, 3, 32, 32).astype('float32') / 255.0
        batch_labels = batch[b'labels']

        images.append(batch_images)
        labels.extend(batch_labels)

    #combining all batches into a single dataset & stacking images vertically
    images = np.vstack(images)
    return images, labels

#doing the same thing for testing data
def load_test_batch(batch_folder):

    batch_file = os.path.join(batch_folder, "test_batch")
    with open(batch_file, 'rb') as f:
        batch = pickle.load(f, encoding = 'bytes')

    images = batch[b'data'].reshape(10000, 3, 32, 32).astype('float32') / 255.0
    labels = batch[b'labels']
    return images, labels

#putting functions in action
batch_folder = "/Users/ryannyathi/Documents/CompIntelligence/Cifar_10_data"  # Path to the folder where the batches are stored
train_images, train_labels = load_cifar_batches(batch_folder)
test_images, test_labels = load_test_batch(batch_folder)



#converting to pytorch tensor

train_images_tensor = torch.from_numpy(train_images)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)

#doing the same for testing data
test_images_tensor = torch.from_numpy(test_images)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.long)


#creating dataloaders
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

#checking the output
print(f"Train loader batches: {len(train_loader)}, Test loader batches: {len(test_loader)}")





Train loader batches: 782, Test loader batches: 157


In [3]:
def evaluate_model(model, test_loader):
    model.eval()  #setting model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  #disabling gradient for evaluation
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)  #getting predicted class
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
   
    return (100 * correct / total)

In [4]:
class SimpleCNN(nn.Module):
    def __init__(self):

        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10) #final layer 128 neurons mapped to 10 output classes

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8) #flattening the array to create a vector for fc layers
        x = torch.relu(self.fc1(x))#passing flattened vector through fc1 with relu activation
        x = self.fc2(x)
        return x

model = SimpleCNN()
model.load_state_dict(torch.load("simple_cnn_model.pth"))

/var/folders/7g/5rjf0wsd6179hwk815vh2rcm0000gn/T/ipykernel_1818/3084219500.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("simple_cnn_

<All keys matched successfully>

In [5]:
# Freezing the last layers 
for name, param in model.named_parameters():
    if(not 'fc2' in name):
        param.requires_grad = False
#Randomising last layer weights
init.uniform_(model.fc2.weight, a=-0.1, b=0.1)
init.uniform_(model.fc2.bias, a=-0.1, b=0.1)

# Extracting weights and biases from the last layer
last_layer_weights = model.fc2.weight.data.numpy().flatten()
last_layer_biases = model.fc2.bias.data.numpy().flatten()

# Combining into a single array for optimization
params = np.concatenate([last_layer_weights, last_layer_biases])
criterion = nn.CrossEntropyLoss()

initial_accuracy = evaluate_model(model,test_loader)

In [6]:
# Fitness function 
def fitness_function(params):
    # Split weights and biases
    num_weights = model.fc2.weight.numel()
    weights = params[:num_weights].reshape(model.fc2.weight.shape)
    biases = params[num_weights:]

    # Updating the model's last layer weights and biases
    model.fc2.weight.data = torch.tensor(weights, dtype=torch.float32)
    model.fc2.bias.data = torch.tensor(biases, dtype=torch.float32)

    # Evaluating the model's accuracy 
    
    accuracy = evaluate_model(model, test_loader)
    return 100 - accuracy


In [7]:
def Differential_Evolution(fitness_function, bounds, population_size, Scaling_Factor, Crossover_Rate, max_iter):
    # Initialize population within bounds
    n = model.fc2.weight.numel() + model.fc2.bias.numel()

    population = np.random.uniform(low=bounds[0], high=bounds[1], size=(population_size, n))
    fitness_values = np.array([fitness_function(ind) for ind in population])

    for iteration in range(max_iter):
        for i in range(population_size):
            # Selecting three random distinct indices
            indices = np.random.choice(np.delete(np.arange(population_size), i), 3, replace=False)
            a, b, c = population[indices]

            # Mutation
            mutant_vector = np.clip(a + Scaling_Factor * (b - c), bounds[0], bounds[1])

            # Crossover
            crossover_mask = np.random.rand(n) <= Crossover_Rate
            trial_vector = np.where(crossover_mask, mutant_vector, population[i])

    
            # Selection
            trial_fit = fitness_function(trial_vector)
            if trial_fit < fitness_values[i]:
                population[i] = trial_vector
                fitness_values[i] = trial_fit
        # Evaluate accuracy
        
        accuracy = evaluate_model(model, test_loader)  
        print(f"Accuracy after iteration {iteration + 1}: {accuracy:.2f}%")

    # Return the best solution and its fitness
    best_index = fitness_values.argmin()
    return population[best_index], fitness_values[best_index]


In [8]:
print(f'Initial Accuracy: {initial_accuracy:.2f}%')
result = Differential_Evolution(fitness_function=fitness_function,bounds=(-1, 1),population_size=10, Scaling_Factor=0.9,Crossover_Rate=0.7,max_iter=1000)



Initial Accuracy: 7.38%
Accuracy after iteration 1: 10.58%
Accuracy after iteration 2: 14.86%
Accuracy after iteration 3: 10.39%
Accuracy after iteration 4: 16.65%
Accuracy after iteration 5: 12.47%
Accuracy after iteration 6: 15.87%
Accuracy after iteration 7: 15.01%
Accuracy after iteration 8: 15.00%
Accuracy after iteration 9: 11.81%
Accuracy after iteration 10: 11.02%
Accuracy after iteration 11: 15.29%
Accuracy after iteration 12: 15.11%
Accuracy after iteration 13: 15.72%
Accuracy after iteration 14: 19.44%
Accuracy after iteration 15: 19.23%
Accuracy after iteration 16: 20.99%
Accuracy after iteration 17: 24.41%
Accuracy after iteration 18: 21.39%
Accuracy after iteration 19: 25.32%
Accuracy after iteration 20: 19.89%
Accuracy after iteration 21: 23.09%
Accuracy after iteration 22: 19.44%
Accuracy after iteration 23: 18.66%
Accuracy after iteration 24: 20.66%
Accuracy after iteration 25: 23.03%
Accuracy after iteration 26: 22.94%
Accuracy after iteration 27: 16.93%
Accuracy afte

KeyboardInterrupt: 